<a href="https://colab.research.google.com/github/contatofe/Estudos-e-Desafios/blob/main/Vis%C3%A3o%20Computacional%20-%20IA%20Expert/Reconhecimento%20de%20gestos%20-%20IA%20Expert/Reconhecimento_de_gestos_AI_Expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importações

import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np

In [2]:
# Montando o drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Carregando a rede neural pré treinada

network = cv2.dnn.readNetFromCaffe('/content/drive/MyDrive/Arquivos - Dados/Curso - visão computacional/Weights/pose_deploy_linevec_faster_4_stages.prototxt',
                                   '/content/drive/MyDrive/Arquivos - Dados/Curso - visão computacional/Weights/pose_iter_160000.caffemodel')

In [19]:
# Previsão de pontos corporais

video = '/content/drive/MyDrive/Arquivos - Dados/Curso - visão computacional/Videos/gesture1.mp4'
captura = cv2.VideoCapture(video)
conected, frame = captura.read()
print(conected, frame.shape)

connection_points = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1, 14],
                  [14,8], [8,9], [9,10], [14,11], [11,12],[12,13]]

True (1080, 808, 3)


In [15]:
# Funções

def check_arms(points):

  head, right_hand, left_hand = 0, 0, 0

  for i, point in enumerate(points):
    if i == 0:
      head = point[1]
    elif i == 4:
      right_hand = point[1]
    elif i == 7:
      left_hand = point[1]

  if right_hand < head and left_hand < head:
    return True
  else:
    return False

def check_legs(points):
  left_hip, right_hip = 0, 0
  left_foot, right_foot = 0, 0

  for i, point in enumerate(points):
    if i == 11:
      left_hip = point[0]
    elif i == 8:
      right_hip = point[0]
    elif i == 13:
      left_foot = point[0]
    elif i == 10:
      right_foot = point[0]

  if (left_foot > left_hip) and (right_foot < right_hip):
    return True
  else:
    return False

In [20]:
# Previsão de gestos

results = '/content/gesture1_result.mp4'
save_video = cv2.VideoWriter(results, cv2.VideoWriter_fourcc(*'XVID'), 10, (frame.shape[1], frame.shape[0]))

threshold = 0.1
while cv2.waitKey(1) < 0:
  conected, frame = captura.read()

  if not conected:
    break

  img_blob = cv2.dnn.blobFromImage(image = frame, scalefactor = 1.0 / 255, size = (256, 256))
  network.setInput(img_blob)
  output = network.forward()

  width_position = output.shape[2]
  height_position = output.shape[3]

  point_number = 15
  points = []

  for i in range(point_number):
    confience_map = output[0, i, :, :]
    _, confidence, _, point = cv2.minMaxLoc(confience_map)
    x = int((frame.shape[1] * point[0]) / width_position)
    y = int((frame.shape[0] * point[1]) / height_position)

    if confidence > threshold:
      cv2.circle(frame, (x, y), 5, (0,255,0), thickness = -1)
      cv2.putText(frame, "{}".format(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, .7, (0, 0, 255))
      points.append((x, y))
    else:
      points.append(None)

  for connection in connection_points:
    parteA = connection[0]
    parteB = connection[1]
    if points[parteA] and points[parteB]:
      cv2.line(frame, points[parteA], points[parteB], (255,0,0))

  if check_arms(points) == True and check_legs(points) == True:
    cv2.putText(frame, 'Completo', (50,200), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0))

  save_video.write(frame)
save_video.release()
print('Processamento concluído!')

Processamento concluído!
